In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 https://cloud.r-project.

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-08-01 01:08:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.71MB/s    in 0.2s    

2022-08-01 01:08:11 (5.71 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


# Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [13]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [15]:
#filter the dataset for total_votes greater than or equal to 20
votes_20_df = df.filter("total_votes >= 20")
votes_20_df.show(5)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [17]:
votes_20_count = votes_20_df.count()
votes_20_count

18739

In [16]:
#create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_50_df = votes_20_df.filter("helpful_votes/total_votes>=.50")
helpful_votes_50_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [18]:
# Total number of Helpful reviews count 
helpful_votes_50_count = helpful_votes_50_df.count()
helpful_votes_50_count

18155

In [24]:
#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
yes_vine_review_df = helpful_votes_50_df.filter(helpful_votes_50_df["vine"] == "Y")
yes_vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35119071|R2BQOD1R0228FN|B00H2RSA88|     405483618|Sleep Innovations...|       Furniture|          3|           17|         26|   Y|                N|An okay product. ...|Three-stars is co...| 2015-08-27|
|         US|   44737123| RC31RUPFOHBHQ|B0125QZ50G|     350975212|Zinus Viscolatex ...|       Furniture|          4|    

In [22]:
#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
no_vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
no_vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [26]:
#Total Number of Reviews PAID and 'helpful
yes_vine_review_count = yes_vine_review_df.count()
yes_vine_review_count

136

In [27]:
#Total Number of Reviews NON-PAID and 'helpful'
no_vine_review_count = no_vine_review_df.count()
no_vine_review_count

18019

In [29]:
# Number of 5-star Reviews for above Paid dataframes
stars5_yes_vine_df = yes_vine_review_df.filter(no_vine_review_df["star_rating"]=="5")
stars5_yes_vine_df.count()

74

In [30]:
# Number of 5-star Reviews for above Non-Paid dataframes
stars5_no_vine_df = no_vine_review_df.filter(no_vine_review_df["star_rating"]=="5")
stars5_no_vine_df.count()


8482

In [31]:
# Percentage of 5-star Reviews for above Paid dataframes
stars5_yes_vine_pct = (stars5_yes_vine_df.count()/yes_vine_review_count)
stars5_yes_vine_pct

0.5441176470588235

In [34]:
# Percentage of 5-star Reviews for above Non-Paid dataframes
stars5_no_vine_pct = (stars5_no_vine_df.count()/no_vine_review_count)
stars5_no_vine_pct


0.47072534546867195

In [38]:
# Total number of ALL reviews
total_reviews_count = df.count()
print("Total Number of Furniture Reviews: %f" % total_reviews_count)
print("Total Number of 'Helpful' Furniture Reviews: %f" % helpful_votes_50_count)



Total Number of Furniture Reviews: 792113.000000
Total Number of 'Helpful' Furniture Reviews: 18155.000000


In [40]:
#Total number of ALL 5-star reviews
stars5_df = df.filter(df["star_rating"] == '5')
stars5_df.show(5)




+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|            0|          0|   N|                Y|          Five Stars|          Great item| 2015-08-31|
|         US|    1272331|R1AIMEEPYHMOE4|B0030MPBZ4|     124663823|Bathroom Vanity T...|       Furniture|          5|    

In [42]:

stars5_count = stars5_df.count()
stars5_count

447716

In [43]:
#Total number of 5-star HELPFUL reviews
stars5_helpful_df = helpful_votes_50_df.filter(helpful_votes_50_df["star_rating"] == '5')
stars5_helpful_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [44]:
#Total Number of 5-Star 'Helpful' Furniture Reviews
stars5_helpful_count = helpful_votes_50_df.count()
stars5_helpful_count

18155

### Percentage of ALL 5-star reviews paid vs non-paid


In [45]:
# Number of 5-Star Furniture Reviews PAID
yes_vine_star5_all_df = stars5_df.filter(stars5_df["vine"] == "Y")
yes_vine_star5_all_count = yes_vine_star5_all_df.count()
yes_vine_star5_all_count

1356

In [51]:
# Percent of 5-Star Furniture Reviews PAID

yes_vine_star5_all_pct = (yes_vine_star5_all_count/stars5_count)
yes_vine_star5_all_pct

0.0030287056973617205

In [49]:
# Number of 5-Star Furniture Reviews NON-PAID
no_vine_star5_all_df = stars5_df.filter(stars5_df["vine"] == "N")
no_vine_star5_all_count = no_vine_star5_all_df.count()
no_vine_star5_all_count

446360

In [52]:
# Percent of 5-Star Furniture Reviews NON-PAID

no_vine_star5_all_pct = (no_vine_star5_all_count/stars5_count)
no_vine_star5_all_pct

0.9969712943026383

### Percentage of HELPFUL 5-star reviews paid vs non-paid

In [59]:
# 5-star Paid "helpful" reviews count
yes_vine_star5_helpful_df = stars5_helpful_df.filter(stars5_helpful_df["vine"] == "Y")
yes_vine_star5_helpful_count = yes_vine_star5_helpful_df.count()

yes_vine_star5_helpful_count


74

In [61]:
# Percent of 5-Star,'Helpful'Furniture Reviews PAID
yes_vine_star5_help_pct = (yes_vine_star5_helpful_count/stars5_helpful_count)

yes_vine_star5_help_pct

0.004076012117873864

In [62]:
# 5-star NON-Paid "helpful" reviews count
no_vine_star5_helpful_df = stars5_helpful_df.filter(stars5_helpful_df["vine"] == "N")
no_vine_star5_helpful_count = no_vine_star5_helpful_df.count()

no_vine_star5_helpful_count


8482

In [63]:
# Percent of 5-Star,'Helpful'Furniture Reviews PAID
no_vine_star5_help_pct = (no_vine_star5_helpful_count/stars5_helpful_count)

no_vine_star5_help_pct

0.4671991187000826